# Comparing implemented methods
## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline
from sklearn.model_selection import KFold

from color_correction.errors import CIELABDE, CIEDE2000
from color_correction.regressions import LCC, PCC, RPCC

## Loading [SFU's surface reflectance dataset](https://www2.cs.sfu.ca/~colour/data/colour_constancy_synthetic_test_data/)
Sorry for magic numbers. We need equal ranges of data that's why we skip some fields
sometimes.

In [2]:
sfu_spd = np.loadtxt("data/reflect_db.reflect", dtype=np.float64).reshape([-1,101])[:, 5:86:]
data_size = sfu_spd.shape[0]

interp_wavelengths = np.arange(400, 721)
interpolator = make_interp_spline(interp_wavelengths[::4], sfu_spd, k=1, axis=1)
sfu_spd = interpolator(interp_wavelengths).reshape(data_size, -1, 1)

sfu_spd.shape

(1995, 321, 1)

## Loading (Nikon D5100 spectral sensivity function)[https://github.com/butcherg/ssf-data/blob/master/Nikon/D5100/camspec/Nikon_D5100.csv]

In [3]:
nikon_ssf = np.loadtxt("data/Nikon_D5100.csv", dtype=np.float64, delimiter=",", usecols=(1,2,3))

interpolator = make_interp_spline(interp_wavelengths[::10], nikon_ssf, k=1, axis=0)
nikon_ssf = interpolator(interp_wavelengths)

nikon_ssf.shape

(321, 3)

## Loading (XYZ color matching function)[https://cie.co.at/datatable/cie-1931-colour-matching-functions-2-degree-observer]

In [4]:
xyz_cmf = np.loadtxt("data/CIE_xyz_1931_2deg.csv", dtype=np.float64, delimiter=",", usecols=(1,2,3))[40:361]
xyz_cmf.shape

(321, 3)

## Loading [D65 spectral poower distribution](https://cie.co.at/datatable/cie-standard-illuminant-d65)

In [5]:
d65_spd = np.loadtxt("data/CIE_std_illum_D65.csv", dtype=np.float64, delimiter=",", usecols=(1))[100:421].reshape(-1, 1)
d65_spd.shape

(321, 1)

## Calculating RGB RAW and CIE XYZ coordinates

In [6]:
rgb = np.trapz(sfu_spd * nikon_ssf * d65_spd, dx=1, axis=1)
xyz = np.trapz(sfu_spd * xyz_cmf * d65_spd, dx=1, axis=1)
print(rgb.shape, xyz.shape)

(1995, 3) (1995, 3)


## Calculating white point

In [7]:
white_point = np.trapz(xyz_cmf * d65_spd, dx=1, axis=0)
white_point

array([10033.22810699, 10565.8810844 , 11469.06421026])

## Performance on fixed exposure

In [8]:
methods = {"linear": LCC(),
           "poly 2 deg.": PCC(degree=2, loss="mse"),
           "root poly 2 deg.": RPCC(degree=2, loss="mse"),
           "poly opt. 2 deg.": PCC(degree=2, loss="cielabde"),
           "root poly opt. 2 deg.": RPCC(degree=2, loss="cielabde")}
    
def get_perf(rgb, xyz, white_point, error_func):
    global methods
    n_splits = 5
    error_statistics = np.zeros((len(methods), 4), dtype=np.float64)

    kf = KFold(n_splits=n_splits, random_state=0, shuffle=True)
    for train_index, test_index in kf.split(rgb):
        rgb_train, rgb_test = rgb[train_index], rgb[test_index]
        xyz_train, xyz_test = xyz[train_index], xyz[test_index]

        batch_statistics = np.zeros_like(error_statistics)
        for j, model in enumerate(methods.values()):
            model.fit(rgb_train, xyz_train, white_point)
            errors = error_func(model.predict(rgb_test), xyz_test, white_point)

            batch_statistics[j] = np.array([np.mean(errors),
                                            np.max(errors),
                                            np.median(errors),
                                            np.percentile(errors, 95)])

        error_statistics += batch_statistics

    error_statistics /= 5
    
    return pd.DataFrame({"Method": methods.keys(),
              "Mean": error_statistics[:, 0],
              "Max": error_statistics[:, 1],
              "Median": error_statistics[:, 2],
              "95%": error_statistics[:, 3]}).style.hide()

### CIELAB Delta E

In [9]:
get_perf(rgb, xyz, white_point, CIELABDE)

/home/denis/work/iitp/color_correction/color_correction/convert.py:11: RuntimeWarning: invalid value encountered in power
  return (t > delta ** 3) * (t ** (1/3)) + (t <= delta ** 3) * (1/3 * t * delta ** (-2) + 4/29)


Method,Mean,Max,Median,95%
linear,1.589252,17.758626,0.889065,5.043187
poly 2 deg.,1.263581,13.124550,0.753671,3.683078
root poly 2 deg.,nan,nan,nan,nan
poly opt. 2 deg.,1.144871,7.285970,0.766822,3.400068
root poly opt. 2 deg.,nan,nan,nan,nan


### CIE Delta E 2000

In [10]:
get_perf(rgb, xyz, white_point, CIEDE2000)

/home/denis/work/iitp/color_correction/color_correction/convert.py:11: RuntimeWarning: invalid value encountered in power
  return (t > delta ** 3) * (t ** (1/3)) + (t <= delta ** 3) * (1/3 * t * delta ** (-2) + 4/29)


Method,Mean,Max,Median,95%
linear,0.905658,6.857275,0.651576,2.522678
poly 2 deg.,0.752311,4.148586,0.543090,2.092545
root poly 2 deg.,nan,nan,nan,nan
poly opt. 2 deg.,0.721213,3.618652,0.533472,1.907736
root poly opt. 2 deg.,nan,nan,nan,nan


### Different exposures

In [11]:
def exposure_test(rgb, xyz, white_point, model, exposure):
    res = model.predict(rgb * exposure)
    return CIELABDE(res, xyz * exposure, white_point * exposure)

n_splits = 5
exposures = [0.2, 0.5, 1, 2, 5]
error_statistics = np.zeros((len(methods), len(exposures)), dtype=np.float64)

kf = KFold(n_splits=n_splits, random_state=0, shuffle=True)
for train_index, test_index in kf.split(rgb):
    rgb_train, rgb_test = rgb[train_index], rgb[test_index]
    xyz_train, xyz_test = xyz[train_index], xyz[test_index]

    batch_statistics = np.zeros_like(error_statistics)
    for j, model in enumerate(methods.values()):
        model.fit(rgb_train, xyz_train, white_point)

        for k, exposure in enumerate(exposures):
            errors = exposure_test(rgb_test, xyz_test, white_point, model, exposure)
            batch_statistics[j][k] = np.mean(errors)

    error_statistics += batch_statistics

error_statistics /= n_splits

df_tmp = pd.DataFrame(columns=exposures,
             data=error_statistics)
df_tmp.insert(0, "Methods", methods.keys())
df_tmp.style.hide()

/home/denis/work/iitp/color_correction/color_correction/convert.py:11: RuntimeWarning: invalid value encountered in power
  return (t > delta ** 3) * (t ** (1/3)) + (t <= delta ** 3) * (1/3 * t * delta ** (-2) + 4/29)


Methods,0.200000,0.500000,1.000000,2.000000,5.000000
linear,1.589252,1.589252,1.589252,1.589252,1.589252
poly 2 deg.,1.611825,1.425788,1.263581,nan,nan
root poly 2 deg.,nan,nan,nan,nan,nan
poly opt. 2 deg.,1.425730,1.279574,1.144871,nan,nan
root poly opt. 2 deg.,nan,nan,nan,nan,nan
